In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchnlp.datasets import imdb_dataset
from torchnlp.datasets import penn_treebank_dataset

In [13]:
def generate_mask(sequence_length, batch_size=None,is_present=0.7):
    """
    e.g.
    returns: [1, 1, 0, 1, 0, 1]
    """
    if batch_size is not None:
        mask = np.random.binomial(1, is_present, size=(batch_size, sequence_length))
    elif batch_size is None:
        mask = np.random.binomial(1, is_present, size=(sequence_length,))
    return torch.from_numpy(mask).long()

In [14]:
def transform_input_with_is_missing_token(inputs, targets_present, masked_value):
    """
    e.g. 
        inputs = [a, b, c, d, e]
        targets = [b, c, d, e, f]
        targets_present = [1, 0, 1, 0, 1]
        masked_value = <m>
        
    then,
        transformed_input = [a, b, <m>, d, <m>]
        
    Args:
        inputs: tensor with shape [batch_size, sequence_length] with tokens
        targets_present: tensor with shape [batch_size, sequence_length] with 1. representing presence of a word of dtype=torch.long
        
    from github.com/tensorflow/models/blob/master/research/maskgan
    """
    mask = torch.cat((torch.zeros((inputs.size()[0], 1), dtype=torch.long), targets_present[:,:-1]), dim=1)
    transformed_input = inputs * mask + masked_value * torch.ones(inputs.size(), dtype=torch.long) * (1 - mask)
    return transformed_input

In [15]:
"""
e.g.
batch_size = 3
sequence_length = 4


input_ = torch.from_numpy(np.arange(1, 13).reshape(3, 4)).long()

present = generate_mask(3, 4)
transformed_input = transform_input_with_is_missing_token(input_, present, 57)
print(present, '\n', transformed_input)"""

"\ne.g.\nbatch_size = 3\nsequence_length = 4\n\n\ninput_ = torch.from_numpy(np.arange(1, 13).reshape(3, 4)).long()\n\npresent = generate_mask(3, 4)\ntransformed_input = transform_input_with_is_missing_token(input_, present, 57)\nprint(present, '\n', transformed_input)"

In [5]:
class GeneratorEncoder(nn.Module): 
    def __init__(self, input_size, hidden_size):
        super(GeneratorEncoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size) # based on GRU just for now

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [6]:
class GeneratorDecoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(GeneratorDecoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [7]:
test = penn_treebank_dataset(test=True)
print(*test[:50])

no it was n't black monday </s> but while the new york stock exchange did n't fall apart friday as the dow jones industrial average plunged N points most of it in the final hour it barely managed to stay this side of chaos </s> some circuit breakers installed after


In [9]:
len(test)

82430

In [8]:
gen_encoder = GeneratorEncoder()

TypeError: __init__() missing 2 required positional arguments: 'input_size' and 'hidden_size'